## Steps outline
1. Start from lvk posterior samples
2. Randomly sample 100 samples (they all have the same probability) without replacement
3. Run GWFish on the sub-sample of lvk posterior samples
4. Take as SNR the average over the realizations
5. Take as fisher samples the averga over the fisher samples
6. Take as prior+fisher samples the average of the prior+fisher samples
7. Calculate the 90% CL from the averaged samples
8. Compare to the lvk samples

In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

from tools import *
import numpy as np
import pickle

import GWFish.modules as gw

In [2]:
PATH_TO_LVK_DATA = '/Users/ulyana/Documents/Paper2_Review/GWTC_data/'
PATH_TO_POSTERIORS_AVERAGED = '/Users/ulyana/Documents/Paper2_Review/results/posterior_samples_averaged/'
PATH_TO_GWFISH_AVERAGED = '/Users/ulyana/Documents/Paper2_Review/results/gwfish_averaged/'
PATH_TO_INFO = '/Users/ulyana/Documents/Paper2_Review/results/info/'
PATH_TO_LVK_SAMPLES = '/Users/ulyana/Documents/Paper2_Review/results/posterior_samples/lvk_samples_snr/'
PATH_TO_YAML = '/Users/ulyana/Documents/Paper2_Review/yamls/'
PATH_TO_PSD = '/Users/ulyana/Documents/Paper2_Review/psds/'
PATH_TO_LVK_SUB_SAMPLES = '/Users/ulyana/Documents/Paper2_Review/results/posterior_samples/lvk_sub_samples/'

In [3]:
waveform = 'IMRPhenomXPHM'
params = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'network_optimal_snr']

In [4]:
events_list = np.loadtxt(PATH_TO_INFO + 'events_wf_median_priors_%s.txt' %waveform, dtype=str)
with open(PATH_TO_INFO + 'detectors_dictionary.pkl', 'rb') as f:
    detectors = pickle.load(f)

In [5]:
results_folder = 'gwfish_averaged'

In [6]:
fisher_parameters = ['chirp_mass', 'mass_ratio', 'luminosity_distance', 'dec', 'ra', 'theta_jn', 'psi', 'phase', 
            'geocent_time', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl']

In [7]:
np.random.seed(42)

In [ ]:
for event in events_list:
        name_tag = 'BBH_%s' %event

        detectors_list = detectors[event]
        detectors_event = []
        for j in range(len(detectors_list)):
            detectors_event.append(detectors_list[j])
        networks = np.linspace(0, len(detectors_event) - 1, len(detectors_event), dtype=int)
        list_all_networks = []
        for i in range(len(networks)):
            list_all_networks.append([networks[i].tolist()])
        list_all_networks.append(networks.tolist()) 
        networks = str(list_all_networks)

        detectors_ids = np.array(detectors_event)
        networks_ids = json.loads(networks)
        ConfigDet = os.path.join(PATH_TO_YAML + event + '.yaml')

        # get sub_sample of injections
        lvk_data = h5py.File(PATH_TO_LVK_DATA + event + '.h5', 'r') 
        samples_lvk = {}
        for l in range(len(params)):
            samples_lvk[params[l]] = lvk_data['C01:IMRPhenomXPHM']['posterior_samples'][params[l]]
        samples_lvk = pd.DataFrame(samples_lvk)
        # get sub_samples
        np.random.seed(42)
        indexes = np.random.choice(samples_lvk.shape[0], 30, replace=False)
        sub_samples_lvk = samples_lvk.iloc[indexes]
        # save for record
        sub_samples_lvk.to_hdf(PATH_TO_LVK_SUB_SAMPLES + event + '.h5', key='samples', mode='w')

        if not os.path.exists(PATH_TO_GWFISH_AVERAGED + event):
            os.makedirs(PATH_TO_GWFISH_AVERAGED + event)

        network = gw.detection.Network(detectors_ids, detection_SNR=(0., 0.), config=ConfigDet)
        gw.fishermatrix.analyze_and_save_to_txt(network = network,
                                        parameter_values  = sub_samples_lvk,
                                        fisher_parameters = fisher_parameters, 
                                        sub_network_ids_list = networks_ids,
                                        population_name = name_tag,
                                        waveform_model = waveform,
                                        save_path = PATH_TO_GWFISH_AVERAGED + event +'/',
                                        save_matrices = True,
                                        decimal_output_format='%.15f')
